In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler


C:\miniconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\miniconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\miniconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\miniconda\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Futu

In [319]:
import tensorflow as tf
from tensorflow.python.keras.layers import  Input,  Dense, LSTM, Concatenate, Dropout, Bidirectional
from tensorflow.python.keras.models import Model
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
plt.style.use('fivethirtyeight')

In [24]:
wind_columns = [ 'guitrancourt_Speed(m/s)', 'guitrancourt_Direction (deg N)',
       'lieusaint_Speed(m/s)', 'lieusaint_Direction (deg N)',
       'lvs-pussay_Speed(m/s)', 'lvs-pussay_Direction (deg N)',
       'parc-du-gatinais_Speed(m/s)', 'parc-du-gatinais_Direction (deg N)',
       'arville_Speed(m/s)', 'arville_Direction (deg N)',
       'boissy-la-riviere_Speed(m/s)', 'boissy-la-riviere_Direction (deg N)',
       'angerville-1_Speed(m/s)', 'angerville-1_Direction (deg N)',
       'angerville-2_Speed(m/s)', 'angerville-2_Direction (deg N)',
       'guitrancourt-b_Speed(m/s)', 'guitrancourt-b_Direction (deg N)',
       'lieusaint-b_Speed(m/s)', 'lieusaint-b_Direction (deg N)',
       'lvs-pussay-b_Speed(m/s)', 'lvs-pussay-b_Direction (deg N)',
       'parc-du-gatinais-b_Speed(m/s)', 'parc-du-gatinais-b_Direction (deg N)',
       'arville-b_Speed(m/s)', 'arville-b_Direction (deg N)',
       'boissy-la-riviere-b_Speed(m/s)',
       'boissy-la-riviere-b_Direction (deg N)', 'angerville-1-b_Speed(m/s)',
       'angerville-1-b_Direction (deg N)', 'angerville-2-b_Speed(m/s)',
       'angerville-2-b_Direction (deg N)']
wind_angle_columns = ['guitrancourt_Direction (deg N)','lieusaint_Direction (deg N)','lvs-pussay_Direction (deg N)',
 'parc-du-gatinais_Direction (deg N)','arville_Direction (deg N)','boissy-la-riviere_Direction (deg N)','angerville-1_Direction (deg N)', 
 'angerville-2_Direction (deg N)','guitrancourt-b_Direction (deg N)','lieusaint-b_Direction (deg N)','lvs-pussay-b_Direction (deg N)',
 'parc-du-gatinais-b_Direction (deg N)','arville-b_Direction (deg N)','boissy-la-riviere-b_Direction (deg N)','angerville-1-b_Direction (deg N)','angerville-2-b_Direction (deg N)'

]
wind_speed_columns = set(wind_columns) - set(wind_angle_columns)
wind_speed_columns = list(wind_speed_columns)

In [2]:
df = pd.read_csv("combined_energy_data.csv")

In [4]:
## convert it to datetime
df['datetime'] = pd.to_datetime(df.datetime)
agg_columns =  ["datetime","Wind"]
required_columns = ["Hours"] + agg_columns + [i for i in df.columns[12:]]
df_agg = df[agg_columns].groupby([df.datetime.dt.floor("H")]).agg(["mean","median","max","min","std","sum"])
df_agg.columns = df_agg.columns.map('_'.join)
df_agg = df_agg.reset_index()


,datetime,Wind_mean,Wind_median,Wind_max,Wind_min,Wind_std,Wind_sum
0,2016-12-31 23:00:00,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df_merged = df_agg.merge(df[required_columns], how="left", on="datetime")

In [7]:
df_interpolated = df_merged.interpolate(method="linear")
df_interpolated = df_interpolated[2:]
df_interpolated = df_interpolated.reset_index(drop=True)
df_interpolated.head()

,datetime,Wind_mean,Wind_median,Wind_max,Wind_min,Wind_std,Wind_sum,Hours,Wind,guitrancourt_Speed(m/s),...,parc-du-gatinais-b_Speed(m/s),parc-du-gatinais-b_Direction (deg N),arville-b_Speed(m/s),arville-b_Direction (deg N),boissy-la-riviere-b_Speed(m/s),boissy-la-riviere-b_Direction (deg N),angerville-1-b_Speed(m/s),angerville-1-b_Direction (deg N),angerville-2-b_Speed(m/s),angerville-2-b_Direction (deg N)
0,2017-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,02:00,0.0,1.961667,...,2.505,221.333333,2.516667,221.333333,2.723333,221.833333,2.72,223.5,2.72,223.5
1,2017-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,03:00,0.0,2.063333,...,2.500,217.666667,2.513333,217.666667,2.716667,217.666667,2.70,219.0,2.70,219.0
2,2017-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,04:00,0.0,2.165000,...,2.495,214.000000,2.510000,214.000000,2.710000,213.500000,2.68,214.5,2.68,214.5
3,2017-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,05:00,0.0,2.266667,...,2.490,210.333333,2.506667,210.333333,2.703333,209.333333,2.66,210.0,2.66,210.0
4,2017-01-01 05:00:00,0.0,0.0,0.0,0.0,0.0,0.0,06:00,0.0,2.368333,...,2.485,206.666667,2.503333,206.666667,2.696667,205.166667,2.64,205.5,2.64,205.5


In [101]:
wind_sum_T_plus_18 = df_interpolated.Wind_sum

In [102]:
def get_loss_len(df):
    return df.shape[0] - df.dropna().shape[0]

In [165]:
def get_lag(series,number_of_frame,lead_time=18,title_name="Wind_mean_T_plus_"):
    df = pd.DataFrame()
    for i in range(lead_time,number_of_frame+lead_time):
        df[title_name+ str(-i)] = series.shift(-i-lead_time)
    return df
    

In [104]:
# get wind sum frame data

In [290]:
wind_sum__from_t_frame_5 = get_lag(wind_mean_T_plus_18,5) # T0
wind_sum_lag_29_frame_3 = get_lag(wind_mean_T_plus_18,3,29) #T-29 lag_month
wind_sum_lag_379_frame_3 = get_lag(wind_mean_T_plus_18,3,379) #T-379 lag month
wind_sum_target = get_lag(wind_mean_T_plus_18,1,0)

In [142]:
# get wind speed frame data
wind_speed_columns

['lvs-pussay_Speed(m/s)',
 'guitrancourt_Speed(m/s)',
 'angerville-2-b_Speed(m/s)',
 'guitrancourt-b_Speed(m/s)',
 'parc-du-gatinais-b_Speed(m/s)',
 'arville_Speed(m/s)',
 'angerville-1-b_Speed(m/s)',
 'lieusaint_Speed(m/s)',
 'boissy-la-riviere-b_Speed(m/s)',
 'angerville-1_Speed(m/s)',
 'arville-b_Speed(m/s)',
 'boissy-la-riviere_Speed(m/s)',
 'lvs-pussay-b_Speed(m/s)',
 'angerville-2_Speed(m/s)',
 'lieusaint-b_Speed(m/s)',
 'parc-du-gatinais_Speed(m/s)']

In [216]:
df_wind_speed = df_interpolated[wind_speed_columns]

In [218]:
wind_speed_group_1 = ["guitrancourt_Speed(m/s)","guitrancourt-b_Speed(m/s)","lvs-pussay-b_Speed(m/s)"]#0,1,2
wind_speed_group_2 = list(set(wind_speed_columns) - set(wind_speed_group_1))

In [266]:
df_wind_speed_group_1 = []
for i in wind_speed_group_1:
    df_wind_speed_group_1.append(get_lag(df_wind_speed[i],3,0,i+"_T_plus_"))
    
df_wind_speed_group_1 = pd.concat(df_wind_speed_group_1,axis=1)
    

In [247]:
df_wind_speed_group_2 = []
for i in wind_speed_group_2:
    df_wind_speed_group_2.append(get_lag(df_wind_speed[i],3,-1,i+"_T_plus_"))
df_wind_speed_group_2 = pd.concat(df_wind_speed_group_2,axis=1)

In [249]:
#make new persistance

In [453]:
persistance = np.array(get_lag(wind_mean_T_plus_18,1,18))+np.array(get_lag(wind_mean_T_plus_18,1,19))+np.array(get_lag(wind_mean_T_plus_18,1,30))
persistance /= 3 

# Test train split

In [273]:
lost = get_loss_len(wind_sum_lag_379_frame_3)

In [274]:
lost

760

In [275]:
df_wind_speed_group_2.head() # 2na as i shifted 1

,lieusaint_Speed(m/s)_T_plus_1,lieusaint_Speed(m/s)_T_plus_0,lieusaint_Speed(m/s)_T_plus_-1,angerville-1_Speed(m/s)_T_plus_1,angerville-1_Speed(m/s)_T_plus_0,angerville-1_Speed(m/s)_T_plus_-1,boissy-la-riviere_Speed(m/s)_T_plus_1,boissy-la-riviere_Speed(m/s)_T_plus_0,boissy-la-riviere_Speed(m/s)_T_plus_-1,lieusaint-b_Speed(m/s)_T_plus_1,...,angerville-1-b_Speed(m/s)_T_plus_-1,boissy-la-riviere-b_Speed(m/s)_T_plus_1,boissy-la-riviere-b_Speed(m/s)_T_plus_0,boissy-la-riviere-b_Speed(m/s)_T_plus_-1,arville-b_Speed(m/s)_T_plus_1,arville-b_Speed(m/s)_T_plus_0,arville-b_Speed(m/s)_T_plus_-1,angerville-2_Speed(m/s)_T_plus_1,angerville-2_Speed(m/s)_T_plus_0,angerville-2_Speed(m/s)_T_plus_-1
0,NaN,NaN,1.636667,NaN,NaN,1.78,NaN,NaN,1.773333,NaN,...,2.72,NaN,NaN,2.723333,NaN,NaN,2.516667,NaN,NaN,1.78
1,NaN,1.636667,1.773333,NaN,1.78,1.89,NaN,1.773333,1.886667,NaN,...,2.70,NaN,2.723333,2.716667,NaN,2.516667,2.513333,NaN,1.78,1.89
2,1.636667,1.773333,1.910000,1.78,1.89,2.00,1.773333,1.886667,2.000000,2.766667,...,2.68,2.723333,2.716667,2.710000,2.516667,2.513333,2.510000,1.78,1.89,2.00
3,1.773333,1.910000,2.046667,1.89,2.00,2.11,1.886667,2.000000,2.113333,2.773333,...,2.66,2.716667,2.710000,2.703333,2.513333,2.510000,2.506667,1.89,2.00,2.11
4,1.910000,2.046667,2.183333,2.00,2.11,2.22,2.000000,2.113333,2.226667,2.780000,...,2.64,2.710000,2.703333,2.696667,2.510000,2.506667,2.503333,2.00,2.11,2.22


In [456]:
wind_sum__from_t_frame_5_dropna = wind_sum__from_t_frame_5[2:-lost].reset_index(drop=True)
wind_sum_lag_29_frame_3_dropna  = wind_sum_lag_29_frame_3[2:-lost].reset_index(drop=True)
wind_sum_lag_379_frame_3_dropna  = wind_sum_lag_379_frame_3[2:-lost].reset_index(drop=True)
wind_speed_group_1_dropna  = df_wind_speed_group_1[2:-lost].reset_index(drop=True)
wind_speed_group_2_dropna  = df_wind_speed_group_2[2:-lost].reset_index(drop=True)
wind_sum_target_norm = wind_sum_target_norm[2:-lost]
persistance = persistance[2:-lost]

In [461]:
wind_sum_5 = StandardScaler()
wind_sum_3 = StandardScaler()
wind_sum_1 = StandardScaler()
wind_speed_group1 = StandardScaler()
wind_speed_group2 = StandardScaler()
wind_sum__from_0_frame_5_norm = wind_sum_5.fit_transform(wind_sum__from_t_frame_5_dropna)

wind_sum_lag_29_frame_3_norm = wind_sum_3.fit_transform(wind_sum_lag_29_frame_3_dropna)
wind_sum_lag_379_frame_3_norm = wind_sum_3.transform(wind_sum_lag_29_frame_3_dropna)

persistance_norm = wind_sum_1.fit_transform(persistance)

wind_speed_group_1_norm = wind_speed_group1.fit_transform(wind_speed_group_1_dropna)
wind_speed_group_2_norm = wind_speed_group2.fit_transform(wind_speed_group_2_dropna)

y = wind_sum_1.transform(wind_sum_target)

In [462]:
total_size = wind_sum__from_0_frame_5_norm.shape[0]
split_index = int(total_size*0.7)
print(split_index)

21177


In [502]:
train_wind_sum__from_0_frame_5_norm = wind_sum__from_0_frame_5_norm[:split_index]
test_wind_sum__from_0_frame_5_norm = wind_sum__from_0_frame_5_norm[split_index:]

train_wind_sum_lag_29_frame_3_norm = wind_sum_lag_29_frame_3_norm[:split_index]
test_wind_sum_lag_29_frame_3_norm = wind_sum_lag_29_frame_3_norm[split_index:]


train_wind_sum_lag_379_frame_3_norm = wind_sum_lag_379_frame_3_norm[:split_index]
test_wind_sum_lag_379_frame_3_norm = wind_sum_lag_379_frame_3_norm[split_index:]

train_wind_speed_group_1_norm = wind_speed_group_1_norm[:split_index]
test_wind_speed_group_1_norm  = wind_speed_group_1_norm[split_index:]

train_wind_speed_group_2_norm = wind_speed_group_2_norm[:split_index]
test_wind_speed_group_2_norm  = wind_speed_group_2_norm[split_index:]

train_y = y[:split_index]
test_y = y[split_index:]

train_persistance_norm = persistance_norm[:split_index]
test_persistance_norm = persistance_norm[split_index:]


In [568]:
all_things = [train_wind_sum__from_0_frame_5_norm,test_wind_sum__from_0_frame_5_norm,
 train_wind_sum_lag_29_frame_3_norm,test_wind_sum_lag_29_frame_3_norm,
 train_wind_sum_lag_379_frame_3_norm,test_wind_sum_lag_379_frame_3_norm ,
 train_wind_speed_group_1_norm,test_wind_speed_group_1_norm ,
 train_wind_speed_group_2_norm,test_wind_speed_group_2_norm 
]
all_pad_size =[ i.reshape((*i.shape),1) for i in all_things]

train_feature_input = [all_pad_size[i] for i in range(0,len(all_pad_size),2)]

test_feature_input = [all_pad_size[i] for i in range(0,len(all_pad_size),2)]


In [569]:
def LSTM_feature_recognizer(name,shape,dropout,recurrent_dropout,lr):
    sequence_input = Input(shape=(shape[1],shape[2],), name=name)
    
    sequence_out = (LSTM(int(shape[2]), input_shape=(shape[1],shape[2]),dropout=dropout,recurrent_dropout=recurrent_dropout,kernel_regularizer=regularizers.l2(1e-6),return_sequences=True))(sequence_input)
    sequence_out = (LSTM(int(shape[2]/2+1), dropout=dropout,recurrent_dropout=recurrent_dropout,kernel_regularizer=regularizers.l2(1e-6)))(sequence_out)
    
    preds = Dense(1,activation="tanh")(sequence_out)
    
    model = Model(inputs=sequence_input,outputs=preds)
    optimizer = Adam(lr)
    model.compile(loss="mean_squared_error",optimizer=optimizer)
    
    return model

In [601]:
def backbone_network(models,shapes,persistant_shape,dropout,recurrent_dropout,lr):
    shapes = [i.shape for i in shapes]
    features_input_1 = Input(shape=(shapes[0][1],shapes[0][2],), name="features_input1")
    features_input_2 = Input(shape=(shapes[1][1],shapes[1][2],), name="features_input2")
    features_input_3 = Input(shape=(shapes[2][1],shapes[2][2],), name="features_input3")
    features_input_4 = Input(shape=(shapes[3][1],shapes[3][2],), name="features_input4")
    features_input_5 = Input(shape=(shapes[4][1],shapes[4][2],), name="features_input5")
    features_inputs = [features_input_1,features_input_2,features_input_3,features_input_4,features_input_5]
    persistant_input = Input(shape=(persistant_shape,),name="persistant_input")
    
    result = []
    for i in range(len(models)):
        result.append(models[i](features_inputs[i]))
    x1 = tf.concat(result,axis=1)
    x2 = Dense(6,activation="tanh")(x1)
    x = Concatenate()([persistant_input,x2])
    x = Dense(4,activation="tanh")(x)
    preds = Dense(1,activation="tanh")(x)
    
    
    model = Model(inputs=[features_input_1,features_input_2,features_input_3,features_input_4,features_input_5,persistant_input],outputs=preds)
    optimizer = Adam(lr)
    model.compile(loss="mean_squared_error",optimizer=optimizer)
    
    return model
    
    

In [602]:
len(train_feature_input)

5

In [603]:
spatial_dropout = 0.1
dropout = 0.1
recurrent_dropout = 0.1
learning_rate = 3e-1

#create our first model
model_list = []
for i in range(0,len(train_feature_input)):
    model_list.append(LSTM_feature_recognizer("input",train_feature_input[i].shape,dropout, recurrent_dropout, learning_rate))

In [604]:
model_list[0].summary()

Model: "model_55"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 5, 1)]            0         
_________________________________________________________________
lstm_92 (LSTM)               (None, 5, 1)              12        
_________________________________________________________________
lstm_93 (LSTM)               (None, 1)                 12        
_________________________________________________________________
dense_79 (Dense)             (None, 1)                 2         
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


In [605]:
model = backbone_network(model_list,train_feature_input,train_persistance_norm.shape[1],dropout,recurrent_dropout,learning_rate)

In [606]:
model.summary()

Model: "model_60"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
features_input1 (InputLayer)    [(None, 5, 1)]       0                                            
__________________________________________________________________________________________________
features_input2 (InputLayer)    [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
features_input3 (InputLayer)    [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
features_input4 (InputLayer)    [(None, 9, 1)]       0                                            
___________________________________________________________________________________________

In [607]:
checkpoint_filepath = "D:\AI4ImpactProject2\weights.best.hdf5"

In [608]:
train_feature_input[0].shape

(21177, 5, 1)

In [ ]:
model_checkpoint_callback_no_diff = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
learning_rate_2 = 3e-2
history1 = model.fit([*train_feature_input,train_persistance_norm], train_y, validation_split = .2,epochs = 10, batch_size = 10, verbose = 1 ,callbacks=[model_checkpoint_callback_no_diff])

Train on 16941 samples, validate on 4236 samples
Epoch 1/10
  400/16941 [..............................] - ETA: 24:46 - loss: 0.9173

(21177, 5, 1)